# 📐 03. 셀 서식 & 스타일링

> **목표**: xlwings로 엑셀 셀의 **폰트, 색상, 테두리, 정렬, 숫자 서식, 병합**을 자유자재로 다룬다.
>
> **왜 중요한가?** — 데이터만 넣으면 "날것"이다. 상사에게 보내는 보고서는 **서식이 곧 신뢰도**다.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | 폰트 설정 | 제목/본문 구분, 강조 표시 |
| 2 | 셀 배경색 | 헤더 구분, 상태별 색상 코딩 |
| 3 | 테두리 설정 | 표 형태 만들기 |
| 4 | 정렬 | 가운데/왼쪽/오른쪽 정렬 |
| 5 | 숫자 서식 | 천단위, %, 날짜, 통화 |
| 6 | 셀 병합 | 제목 영역, 카테고리 그룹핑 |
| 7 | 행/열 크기 | 깔끔한 레이아웃 |
| 8 | 🔥 실전 종합 | 보고서 서식 원클릭 완성 |

---
## 🧠 핵심 이론: API 방식 vs 네이티브 방식

xlwings에서 스타일을 설정하는 방법은 **두 가지**입니다:

| 방식 | 문법 | 특징 |
|:---|:---|:---|
| **네이티브** | `ws.range('A1').font.bold = True` | 간단, 자주 쓰는 것만 지원 |
| **API (COM)** | `ws.range('A1').api.Font.Bold = True` | Excel VBA의 모든 기능 사용 가능 |

> 💡 **실무 팁**: 네이티브로 안 되는 건 `.api`를 쓰면 **거의 다 된다**.  
> VBA 매크로 녹화 → 코드 변환이 가장 빠른 학습법!

---
## 0️⃣ 준비: 엑셀 파일 열기

In [ ]:
import xlwings as xw

# 엑셀 앱 실행
app = xw.App(add_book=False)

# 교육이수현황 파일 열기
wb = app.books.open('교육이수현황.xlsx')

# 마케팅팀 시트 선택
ws = wb.sheets['마케팅팀']

print(f'✅ 엑셀 열기 완료: {wb.name}')
print(f'📋 선택된 시트: {ws.name}')

---
## 1️⃣ 폰트 설정

네이티브 방식으로 간단하게 폰트를 설정할 수 있습니다.

In [ ]:
# --- 기본 폰트 설정 (네이티브 방식) ---
cell = ws.range('B2')

cell.font.name = '맑은 고딕'       # 글꼴
cell.font.size = 14                # 크기
cell.font.bold = True              # 굵게
cell.font.italic = False           # 기울임
cell.font.color = (0, 0, 0)        # RGB 검정 (튜플)
# cell.font.color = '#FF0000'      # 16진수 빨강도 가능

print('✅ B2 셀 폰트 설정 완료 - 엑셀에서 확인해보세요!')

### 🎯 실무 패턴: 헤더 행 한번에 꾸미기

범위를 지정하면 반복문 없이도 한번에 여러 셀에 적용됩니다.

In [ ]:
# 헤더 범위를 한번에 설정 → 반복문 불필요!
header = ws.range('A2:D2')

header.font.name = '맑은 고딕'
header.font.size = 11
header.font.bold = True
header.font.color = (255, 255, 255)   # 흰색 글자 (배경색과 대비)

print('✅ A2:D2 헤더 폰트 일괄 적용 완료')

---
## 2️⃣ 배경색 설정

### 🎨 실무에서 자주 쓰는 색상 팔레트

| 용도 | RGB | 색상 |
|:---|:---|:---|
| 헤더 배경 | `(47, 85, 151)` | 🟦 네이비 블루 |
| 짝수 행 | `(221, 235, 247)` | 🔵 연한 파랑 |
| 경고/주의 | `(255, 199, 206)` | 🔴 연한 빨강 |
| 완료/정상 | `(198, 239, 206)` | 🟢 연한 초록 |
| 합계 행 | `(255, 242, 204)` | 🟡 연한 노랑 |

In [ ]:
# 헤더에 네이비 블루 배경 적용
ws.range('A2:D2').color = (47, 85, 151)

print('✅ 헤더 배경색 적용 완료 (네이비 블루)')

In [ ]:
# 줄무늬 효과 (짝수 행에 연한 파랑)
# 실무에서 가독성 향상에 매우 효과적!

for i in range(3, 7):
    if i % 2 == 1:  # 홀수 행 
        ws.range(f'A{i}:D{i}').color = (221, 235, 247)  # 연한 파랑
    # 짝수 행은 배경색 없음 (흰색 유지)

print('✅ 줄무늬 효과 적용 완료')

In [ ]:
# 참고: 색 초기화 (투명으로 되돌리기)
# ws.range('A3:D3').color = None

print('💡 색을 지우려면: ws.range("A3:D3").color = None')

---
## 3️⃣ 테두리 설정 (`.api` 필수)

> ⚠️ 테두리는 네이티브 방식이 없어서 **반드시 `.api`**를 사용합니다.

### 테두리 상수 정리

| 인덱스 | 위치 | 인덱스 | 위치 |
|:---:|:---|:---:|:---|
| 7 | 왼쪽(EdgeLeft) | 10 | 오른쪽(EdgeRight) |
| 8 | 위쪽(EdgeTop) | 11 | 내부 세로(InsideVertical) |
| 9 | 아래쪽(EdgeBottom) | 12 | 내부 가로(InsideHorizontal) |

| LineStyle | 의미 | Weight | 의미 |
|:---:|:---|:---:|:---|
| 1 | 실선 | 1 | 가는선 |
| -4115 | 점선 | 2 | 얇은선 |
| -4118 | 도트 | -4138 | 중간선 |
| | | 4 | 굵은선 |

In [ ]:
# 🎯 실무 함수: 표 전체에 테두리 한방에 치기

def set_border(ws, cell_range, style=1, weight=2):
    """
    표 범위에 테두리를 한번에 설정하는 함수
    
    Parameters:
        ws: 워크시트 객체
        cell_range: 'A1:G10' 형태의 범위 문자열
        style: 선 스타일 (1=실선)
        weight: 선 굵기 (2=얇은선)
    """
    rng = ws.range(cell_range).api
    
    # 바깥 테두리 (7~10)
    for border_id in range(7, 11):
        rng.Borders(border_id).LineStyle = style
        rng.Borders(border_id).Weight = weight
    
    # 내부 테두리 (11~12) - 셀이 2개 이상일 때만
    for border_id in range(11, 13):
        try:
            rng.Borders(border_id).LineStyle = style
            rng.Borders(border_id).Weight = weight
        except:
            pass  # 단일 셀이면 내부 테두리 불필요

print('✅ set_border() 함수 정의 완료')

In [ ]:
# 테두리 적용
set_border(ws, 'A2:D6')

print('✅ A2:D6 범위에 테두리 적용 완료')

---
## 4️⃣ 정렬 설정

### 정렬 상수 정리

| 수평 정렬 | 상수 | 수직 정렬 | 상수 |
|:---|:---:|:---|:---:|
| 왼쪽 | -4131 | 위쪽 | -4160 |
| 가운데 | -4108 | 가운데 | -4108 |
| 오른쪽 | -4152 | 아래쪽 | -4107 |

In [ ]:
# 헤더: 가운데 정렬
ws.range('A2:D2').api.HorizontalAlignment = -4108   # 수평 가운데
ws.range('A2:D2').api.VerticalAlignment = -4108      # 수직 가운데

# 숫자 열: 오른쪽 정렬
ws.range('C3:C6').api.HorizontalAlignment = -4152

# 텍스트 열: 왼쪽 정렬
ws.range('B3:B6').api.HorizontalAlignment = -4131

print('✅ 정렬 적용 완료')

In [ ]:
# 자동 줄바꿈 (셀 내용이 길 때 유용)
ws.range('A2:D2').api.WrapText = True

print('✅ 헤더 자동 줄바꿈 설정 완료')

---
## 5️⃣ 숫자 서식

### 💡 숫자 서식 치트시트

| 서식 코드 | 입력값 | 표시 결과 |
|:---|:---:|:---:|
| `#,##0` | 1234567 | 1,234,567 |
| `0.00%` | 0.856 | 85.60% |
| `yyyy-mm-dd` | 날짜 | 2026-02-23 |
| `#,##0 "원"` | 50000 | 50,000 원 |
| `0.0 "시간"` | 8.5 | 8.5 시간 |
| `[Blue]#,##0;[Red]-#,##0` | -500 | **-500** (빨강) |

In [ ]:
# 교육시간 열에 숫자 서식 적용
ws.range('C3:C6').number_format = '#,##0'   # 천단위 콤마

print('✅ C3:C6 숫자 서식(천단위) 적용 완료')

In [ ]:
# 다양한 숫자 서식 예시 (참고용 - F열에 테스트)
# 직접 테스트하고 싶으면 주석 해제!

# ws.range('H2').value = '서식 테스트'
# ws.range('H3').value = 1234567
# ws.range('H3').number_format = '#,##0'           # → 1,234,567

# ws.range('H4').value = 0.856
# ws.range('H4').number_format = '0.00%'           # → 85.60%

# ws.range('H5').value = 50000
# ws.range('H5').number_format = '#,##0 "원"'      # → 50,000 원

# ws.range('H6').value = 8.5
# ws.range('H6').number_format = '0.0 "시간"'      # → 8.5 시간

print('💡 다양한 서식 예시는 주석 해제 후 실행해보세요!')

---
## 6️⃣ 셀 병합 / 해제

- `merge()` → 셀 병합  
- `unmerge()` → 셀 병합 해제  
- 병합 셀에 값을 넣으려면 **왼쪽 상단 셀**에 입력해야 함

In [ ]:
# 제목 영역 병합 (1행 전체)
ws.range('A1:D1').merge()
ws.range('A1').value = '📊 마케팅팀 교육이수현황'
ws.range('A1').font.size = 16
ws.range('A1').font.bold = True
ws.range('A1').font.name = '맑은 고딕'
ws.range('A1').font.color = (255, 255, 255)
ws.range('A1').color = (47, 85, 151)
ws.range('A1:D1').api.HorizontalAlignment = -4108   # 가운데

print('✅ 제목 병합 및 서식 적용 완료')

In [ ]:
# 참고: 병합 해제하기
# ws.range('A1:D1').unmerge()

# 참고: 카테고리 병합 (같은 부서끼리 묶기)
# ws.range('A3:A5').merge()
# ws.range('A3:A5').api.VerticalAlignment = -4108  # 세로 가운데

print('💡 병합 해제: ws.range("A1:D1").unmerge()')

---
## 7️⃣ 행 높이 / 열 너비

In [ ]:
# 제목 행 높이 설정
ws.range('A1').row_height = 45    # 제목 행
ws.range('A2').row_height = 30    # 헤더 행

print('✅ 행 높이 설정 완료')

In [ ]:
# 열 너비 자동 맞춤 (AutoFit) ← 실무에서 가장 많이 씀!
ws.range('A1:D6').columns.autofit()

print('✅ 열 너비 자동 맞춤 완료')

In [ ]:
# 수동으로 열 너비 지정하기 (필요할 때)
# ws.range('A:A').column_width = 15
# ws.range('B:B').column_width = 12

print('💡 수동 열 너비: ws.range("A:A").column_width = 15')

---
## 🔥 8. 실전 종합 예제: 보고서 서식 한번에 완성

> 아래 `format_report()` 함수를 만들면,  
> **어떤 시트든** 이 함수 하나로 서식을 일괄 적용할 수 있습니다.  
> 이걸 만들어두면 반복 작업이 **영원히** 사라집니다!

In [ ]:
def format_report(ws, title, header_range, data_range):
    """
    보고서 서식을 한번에 적용하는 실전 함수
    
    Parameters:
        ws: 워크시트 객체
        title: 보고서 제목 문자열
        header_range: 헤더 범위 (예: 'A2:D2')
        data_range: 데이터 전체 범위 (예: 'A2:D6')
    """
    
    # header_range에서 열 범위 추출
    first_col = header_range.split(':')[0][0]
    last_col = header_range.split(':')[1][0]
    title_range = f'{first_col}1:{last_col}1'
    
    # 1) 제목 영역
    ws.range(title_range).merge()
    ws.range(f'{first_col}1').value = title
    ws.range(f'{first_col}1').font.size = 16
    ws.range(f'{first_col}1').font.bold = True
    ws.range(f'{first_col}1').font.name = '맑은 고딕'
    ws.range(f'{first_col}1').font.color = (255, 255, 255)
    ws.range(f'{first_col}1').color = (47, 85, 151)
    ws.range(title_range).api.HorizontalAlignment = -4108
    ws.range(f'{first_col}1').row_height = 45
    
    # 2) 헤더 스타일
    ws.range(header_range).font.bold = True
    ws.range(header_range).font.size = 11
    ws.range(header_range).font.name = '맑은 고딕'
    ws.range(header_range).font.color = (255, 255, 255)
    ws.range(header_range).color = (68, 114, 196)
    ws.range(header_range).api.HorizontalAlignment = -4108
    ws.range(header_range).api.VerticalAlignment = -4108
    ws.range(header_range.split(':')[0]).row_height = 30
    
    # 3) 데이터 영역 테두리
    rng = ws.range(data_range).api
    for border_id in range(7, 13):
        try:
            rng.Borders(border_id).LineStyle = 1
            rng.Borders(border_id).Weight = 2
        except:
            pass
    
    # 4) 열 자동 맞춤
    ws.range(data_range).columns.autofit()
    
    print(f'  ✅ "{title}" 서식 적용 완료')


print('✅ format_report() 함수 정의 완료!')
print('💡 이제 아래 셀에서 이 함수를 호출해보세요.')

In [ ]:
# 마케팅팀 시트에 보고서 서식 적용!
format_report(
    ws=ws,
    title='📊 마케팅팀 교육이수현황',
    header_range='A2:D2',
    data_range='A2:D6'
)

print('\n🎉 엑셀에서 결과를 확인해보세요!')

---
## 💾 저장 & 마무리

In [ ]:
# 저장
wb.save()
print('✅ 파일 저장 완료!')

# 엑셀 닫기 (확인 후 실행!)
# app.quit()

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `AttributeError: has no attribute 'api'` | xlwings 버전이 너무 낮음 | `pip install --upgrade xlwings` |
| `pywintypes.com_error` | 엑셀이 닫혀있는 상태에서 실행 | `app = xw.App()` 먼저 실행 |
| 테두리가 안 보임 | LineStyle만 설정하고 Weight 빠뜨림 | Weight도 같이 설정 |
| 색상이 이상함 | RGB 순서가 뒤바뀜 | (R, G, B) 순서 확인 |
| 병합 셀에 값 입력 안 됨 | 병합 범위의 왼쪽 상단 셀에 입력해야 함 | `ws.range('A1').value` 사용 |

---
## 📝 연습 문제

1. `영업1팀` 시트에도 `format_report()` 함수를 적용해 보세요
2. 합계 행에 **노란 배경** + **굵은 글씨**를 적용해 보세요
3. `format_report()` 함수에 **줄무늬 효과**(짝수 행 연한 파랑) 기능을 추가해 보세요

In [ ]:
# 🏋️ 연습 1: 영업1팀 시트에 format_report() 적용
# 아래 코드를 완성해보세요!

# ws2 = wb.sheets['영업1팀']
# format_report(
#     ws=ws2,
#     title='📊 영업1팀 교육이수현황',
#     header_range='???',   # ← 범위를 채워보세요
#     data_range='???'      # ← 범위를 채워보세요
# )

In [ ]:
# 🏋️ 연습 2: 합계 행에 노란 배경 + 굵은 글씨
# 아래 코드를 완성해보세요!

# ws.range('A6:D6').color = (???, ???, ???)  # 연한 노랑 RGB
# ws.range('A6:D6').font.bold = ???

---
## 📌 다음 챕터 예고

> **04. 반복 자동화 & 여러 시트 한번에 처리**  
> `for`문으로 시트를 돌면서 데이터/서식을 일괄 적용하는 방법